<h1> Choose an image to test the model </h1>

In [9]:
import json
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
from tkinter import filedialog
from tkinter import Tk
import os

# Load the trained model
model_path = r"C:\Users\Dell\Desktop\Bachelor\image_classification\misc\runned\arabic letters\2\model.keras"
print("loading the model...")
model = tf.keras.models.load_model(model_path)
print("model loaded successfully")
# Load the class labels
index_to_label_path = r"C:\Users\Dell\Desktop\Bachelor\image_classification\misc\runned\arabic letters\2\index_to_label.json"
with open(index_to_label_path, 'r', encoding='utf-8') as f:
    class_labels = json.load(f)

# Open a file chooser and get the image path
root = Tk()
root.attributes('-topmost', True) # bring the window to the front
root.withdraw() # we don't want a full GUI, so keep the root window from appearing
root.update()
image_path = filedialog.askopenfilename() # show an "Open" dialog box and return the path to the selected file
root.destroy()

# Load the image
img_width, img_height = 128, 128
img = image.load_img(image_path, target_size=(img_width, img_height))

# Display the image
img.show()

# Preprocess the image
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img.astype('float32')/255 


# Use the model to predict the class of the image
predictions = model.predict(img)

# The output will be the probabilities for each class. To get the class with the highest probability, use argmax
predicted_class_index = np.argmax(predictions)
sorted_predictions = sorted(enumerate(predictions[0]), key=lambda x: x[1], reverse=True)
for i, probability in sorted_predictions:
    printed_text = f'({class_labels[str(i)]}) :  {probability:.4f}'
    if(i == predicted_class_index):
        printed_text += ' <---------------------------------------------------'
    print(printed_text)

loading the model...
model loaded successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
(ه) :  0.9993 <---------------------------------------------------
(ا) :  0.0005
(لا) :  0.0001
(ل) :  0.0000
(ط) :  0.0000
(ف) :  0.0000
(و) :  0.0000
(ظ) :  0.0000
(ق) :  0.0000
(م) :  0.0000
(غ) :  0.0000
(ع) :  0.0000
(ر) :  0.0000
(٩) :  0.0000
(ب) :  0.0000
(ض) :  0.0000
(٦) :  0.0000
(٣) :  0.0000
(ي) :  0.0000
(١) :  0.0000
(٥) :  0.0000
(٨) :  0.0000
(ص) :  0.0000
(٠) :  0.0000
(٧) :  0.0000
(س) :  0.0000
(٢) :  0.0000
(ك) :  0.0000
(ن) :  0.0000
(د) :  0.0000
(خ) :  0.0000
(ز) :  0.0000
(ش) :  0.0000
(ح) :  0.0000
(ج) :  0.0000
(٤) :  0.0000
(ت) :  0.0000
(ذ) :  0.0000
(ث) :  0.0000


<h1> Choose a zip file to extract the images </h1>

In [ ]:
import os
import shutil
import zipfile
from tkinter import Button, ttk ,filedialog, Label, TclError, Frame
from tkinterdnd2 import DND_FILES, TkinterDnD
import tkinter.messagebox

def extract_zip(source_path, destination_path, progressbar , drop_label):
    drop_label.config(text="Extracting....")
    drop_label.update()
    if os.path.exists(destination_path):
        shutil.rmtree(destination_path)
    with zipfile.ZipFile(source_path, 'r') as zip_ref:
        files = zip_ref.infolist()
        progressbar['maximum'] = len(files)
        for i, file in enumerate(files):
            zip_ref.extract(file, path=destination_path)
            progressbar['value'] = i
            progressbar.update()
        progressbar['value'] = len(files)
        
        

def drop(event):
    file_path = event.data[1:-1]  # remove the curly braces
    if file_path:
        zip_file_name = os.path.basename(file_path)
        destination_path = os.path.join(os.path.dirname(file_path), os.path.splitext(zip_file_name)[0])
        extract_zip(file_path, destination_path, progressbar, drop_label)
        os.startfile(os.path.dirname(destination_path))

def choose_file_and_extract(root, progressbar, drop_label):
    file_path = filedialog.askopenfilename(filetypes=[('Zip files', '*.zip')])
    if file_path:
        zip_file_name = os.path.basename(file_path)
        destination_path = os.path.join(os.path.dirname(file_path), os.path.splitext(zip_file_name)[0])
        extract_zip(file_path, destination_path, progressbar, drop_label)
        os.startfile(os.path.dirname(destination_path))
        root.destroy()  # Close the window after execution


root = TkinterDnD.Tk()
root.title("Zip Extractor") 
# root.configure(bg='lightblue')
root.geometry('990x540')  # Set the window size to half of your screen size

progressbar = ttk.Progressbar(root, length=990, mode='determinate')
progressbar.pack() 

drop_label = Label(root, text="Drop your file here", font=("Arial", 24))
# drop_label.place(relx=0.5, rely=0.5, anchor='center')
drop_label.pack(pady=100)

choose_button = Button(root, text="Or Choose Manually", command=lambda: choose_file_and_extract(root, progressbar, drop_label), height=2, width=20, font=("Arial", 20), bg='green', fg='white', relief='raised', borderwidth=5)
choose_button.pack()

root.drop_target_register(DND_FILES)
root.dnd_bind('<<Drop>>', drop)

try:
    root.mainloop()
except TclError:
    pass
except Exception as e:
    tkinter.messagebox.showerror("Error", str(e))



<h1> Move images from one folder to another</h1>


In [ ]:
import os
import shutil

def move_files_with_rename(src_folder, dst_folder):
    # Iterate over all files in the source folder
    for filename in os.listdir(src_folder):
        src_file = os.path.join(src_folder, filename)
        dst_file = os.path.join(dst_folder, filename)

        # If a file with the same name exists in the destination folder, rename the file being moved
        if os.path.exists(dst_file):
            base, extension = os.path.splitext(filename)
            filename = f"{base}_changed{extension}"
            dst_file = os.path.join(dst_folder, filename)

        # Move the file
        shutil.move(src_file, dst_file)

# Use the function
src_folder = "C:/Users/Dell/Desktop/test/500/test"
dst_folder = "C:/Users/Dell/Desktop/test/500/train"
move_files_with_rename(src_folder, dst_folder)

<h1> Divide a dataset into train & test folders </h1>

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the source directory and the target directories
source_dir = './data set/all/'
train_dir = './data set/train/'
test_dir = './data set/test/'

# Create the target directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get a list of all the dog and cat image filenames
dog_images = [i for i in os.listdir(source_dir) if 'dog' in i]
cat_images = [i for i in os.listdir(source_dir) if 'cat' in i]

# Split the filenames into training and testing sets
train_dogs, test_dogs = train_test_split(dog_images, test_size=0.25)
train_cats, test_cats = train_test_split(cat_images, test_size=0.25)

# Function to move files
def move_files(files, target_dir):
    for file in files:
        shutil.move(source_dir + file, target_dir + file)

# Move the corresponding files into the appropriate directories
move_files(train_dogs, train_dir)
move_files(test_dogs, test_dir)
move_files(train_cats, train_dir)
move_files(test_cats, test_dir)


<h1> Join folders containing different letter positions into one folder </h1>

In [ ]:
import os
import shutil

root_dir = r"C:\Users\Dell\Desktop\datasets\1500 (also contains paragraphs)\1500 (also contains paragraphs)\isolated_alphabets_per_alphabet"
# dest is beside the root directory
dest_dir = os.path.join(os.path.dirname(root_dir), "joined")

# Get a list of all subdirectories in the root directory
subdirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
print("started")
if(os.path.exists(dest_dir)):
    print("deleting existing root directory...")
    shutil.rmtree(dest_dir)
for subdir in subdirs:
    print("woring in ", subdir)
    if '_' in subdir: # because some subdirectories (numbers) don't have an underscore in their naems and dont need joining
        letter_name = subdir.split('_')[0]
        new_dir = os.path.join(dest_dir, letter_name)
        os.makedirs(new_dir, exist_ok=True)
        files = [f for f in os.listdir(os.path.join(root_dir, subdir))]
        for file in files:
            shutil.copy(os.path.join(root_dir, subdir, file), os.path.join(new_dir, file))
    else:
        shutil.copytree(os.path.join(root_dir, subdir), os.path.join(dest_dir, subdir))
print("done")        

<h1> same as above code but copy folders only not the files they contain </h1>

In [ ]:
import os
import shutil

root_dir = r"C:\Users\Dell\Desktop\datasets\1800 - position categorized\1800 - position categorized\HMBD-v1-master\Dataset"
# dest is beside the root directory
dest_dir = os.path.join(os.path.dirname(root_dir), "joined")

# Get a list of all subdirectories in the root directory
subdirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
print("started")
if(os.path.exists(dest_dir)):
    print("deleting existing root directory...")
    shutil.rmtree(dest_dir)
for subdir in subdirs:
    print("woring in ", subdir)
    if '_' in subdir: # because some subdirectories (numbers) don't have an underscore in their naems and dont need joining
        letter_name = subdir.split('_')[0]
    else :
        letter_name = subdir    
    new_dir = os.path.join(dest_dir, letter_name)
    os.makedirs(new_dir, exist_ok=True)
    shutil.copytree(os.path.join(root_dir, subdir), os.path.join(new_dir, subdir))
print("done")        

<h1> extract all files from directory to its parent directory , for all subdirectories in the dataset </h1>

In [ ]:
# <h1> extract all files from directory to its parent directory , for all subdirectories in the dataset </h1>

import os
import shutil
def join_files_in_subdirs(dest,current_dir):
    if(not os.path.isdir(current_dir)):
        shutil.copy(current_dir, dest)
    else:
        for subdir in os.listdir(current_dir):
            join_files_in_subdirs(dest, os.path.join(current_dir, subdir))
        contains_files_only = all(os.path.isfile(os.path.join(current_dir, child_dir)) for child_dir in os.listdir(current_dir))
        if contains_files_only:
           print ("done with ", current_dir)
           

    
root_dir = r"C:\Users\Dell\Desktop\datasets\1800 - position categorized\joined"
dest_dir = os.path.join(os.path.dirname(root_dir), "joined & merged")
if(os.path.exists(dest_dir)):
    print("deleting existing root directory...")
    shutil.rmtree(dest_dir)
os.makedirs(dest_dir, exist_ok=True)

for subdir in os.listdir(root_dir):
    merging_dest = os.path.join(dest_dir, subdir)
    os.makedirs(merging_dest, exist_ok=True)
    join_files_in_subdirs(merging_dest, os.path.join(root_dir, subdir))
print("done merging ")
# open the parent directory of the root directory upon finishing
os.startfile(os.path.dirname(dest_dir))

    

<h1> Clear kaggle output folder </h1>

In [63]:
import os
import shutil

def clear_directory(dir_path):
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # remove file or symlink
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # remove directory

# Usage:
dir_path = "/kaggle/working/"
clear_directory(dir_path)

normal sorting :  ['1 - Alf', '10 - Raa', '11 - Zin', '12 - Sin', '13 - Shen', '14 - Saad', '15 - Daad', '16 - Tah', '17 - Zah', '18 - Ain', '19 - Gen', '2 - Baa', '20 - Faa', '21 - Qaf', '22 - Kaf', '23 - Lam', '24 - Mem', '25 - Non', '26 - Ha', '27 - Waw', '28 - Yaa', '29 - AlfHamzaAbove', '3 - Taa', '30 - AlfHamzaUnder', '31 - Hamza', '32 - LamAlf', '33 - LamAlfHamza', '34 - LamAlfMad', '35 - zero', '36 - one', '37 - two', '38 - three', '39 - four', '4 - Thaa', '40 - five', '41 - six', '42 - seven', '43 - eight', '44 - nine', '5 - Gem', '6 - Haa', '7 - Khaa', '8 - Dal', '9 - Zal']
sorted by number :  ['1 - Alf', '2 - Baa', '3 - Taa', '4 - Thaa', '5 - Gem', '6 - Haa', '7 - Khaa', '8 - Dal', '9 - Zal', '10 - Raa', '11 - Zin', '12 - Sin', '13 - Shen', '14 - Saad', '15 - Daad', '16 - Tah', '17 - Zah', '18 - Ain', '19 - Gen', '20 - Faa', '21 - Qaf', '22 - Kaf', '23 - Lam', '24 - Mem', '25 - Non', '26 - Ha', '27 - Waw', '28 - Yaa', '29 - AlfHamzaAbove', '30 - AlfHamzaUnder', '31 - Hamza',